<a href="https://colab.research.google.com/github/physicaone/loss_IG/blob/master/%5B210901%5DTrain_and_get_data6_infinity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import torch.utils.data
import torch.nn as nn
from tqdm import tqdm, tnrange
import warnings
warnings.filterwarnings("ignore")
import random
import pickle as pkl
import pandas as pd
from scipy.stats import entropy
import copy
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0

try:
    from google.colab import drive
    drive.mount('/content/drive')
    base='drive/MyDrive'
except:
    if torch.cuda.device_count()>1:
        base='.'
    else:
        base='Google Drive'

if CUDA:
    device='cuda'
else:
    device='cpu'
torch.cuda.is_available()

Mounted at /content/drive


True

In [ ]:
def Ising_model(lattice, T, n0, n_delta, n_sample):
    #n0는 equlibrium에 도달하는데 걸리는 시간
    #n_delta는 correlation이 줄어드는데 걸리는 시간
    #n_sample은 sampling하고 싶어은 샘플의 숫자
    L = len(lattice)
    samples=[]
    equil,count=0,0
    #alpha
    sampled=0 #샘플링된 샘플들의 숫자를 셉니다. 
    while sampled < n_sample: #원하는 갯수만큼 sampling을 해야 하므로 이 부분은 while문이 와야 합니다.
        if equil < n0: #equilibrium에 도달할 때까지는 샘플을 수집해서는 안됩니다. 
            for i in range(L):
                for j in range(L):
                    s = lattice[i,j]
                    neigh = lattice[(i+1)%L, j] + lattice[i,(j+1)%L] + lattice[(i-1)%L,j] + lattice[i,(j-1)%L]
                    DE = 2 * s * neigh 

                    if DE < 0:
                        s *= -1 
                    elif np.random.rand() < np.exp(- DE*1/T):
                        s *= -1 
                    lattice[i,j] = s
            equil += 1;

        else: #이제는 sample을 수집해도 됩니다.
            x = np.random.randint(L)
            y = np.random.randint(L)
            s = lattice[x,y]  
            neigh = lattice[(x+1)%L, y] + lattice[x,(y+1)%L] + lattice[(x-1)%L,y] + lattice[x,(y-1)%L]
            DE = 2 * s * neigh 

            #accept or reject 
            if DE < 0:
                s *= -1 

            elif np.random.rand() < np.exp(- DE*1/T):
                s *= -1 

            lattice[x,y] = s
            count += 1;
            equil+=1; #이건 굳이 없어도 됩니다. 
            if count == n_delta: #correlation이 큰 샘플은 버려야 합니다. 따라서 n_delta라는 step차이를 두고 sampling을 해야 합니다.
                count = 0;
                a=lattice.copy()
                samples.append(np.float32(np.reshape(a,(1,L*L))+1)/2)
                #beta
                sampled+=1; 
                
    return samples

In [ ]:
def decimal_to_binary(integer, n_hid):
    string=bin(int(integer))[2:]
    list0=[float(d) for d in string]
    while len(list0)<n_hid:
        list0=[0.]+list0
    return torch.tensor([list0])

def Ising_energy(v_list):
    L = 3
    E_list=[]
    for n in range(len(v_list)):
        v=v_list[n]
        E = 0
        for i in range(L):
            for j in range(L):
                s = v[i,j]
                neigh = v[(i+1)%L, j] + v[i,(j+1)%L] + v[(i-1)%L,j] + v[i,(j-1)%L] 
                E += -neigh * s
        E_list.append(E/2)
    return np.array(E_list)
    
class RBM(nn.Module):

    def __init__(self, n_vis, n_hid, k):
        """Create a RBM."""
        super(RBM, self).__init__()
        
        self.v_bias = nn.Parameter(torch.zeros(1, n_vis).to(device))
        self.h_bias = nn.Parameter(torch.zeros(1, n_hid).to(device))
        self.Weight = nn.Parameter(std*torch.randn(n_hid, n_vis).to(device))
        self.k = k


    def v2h(self, v):
        return torch.sigmoid(F.linear(v, self.Weight, self.h_bias))

    def h2v(self, h):
        return torch.sigmoid(F.linear(h, self.Weight.t(), self.v_bias))
    
    def Fv(self, v):
        v_term = torch.matmul(v, self.v_bias.t()).view(len(v))
        h_term = torch.sum(F.softplus(F.linear(v, self.Weight, self.h_bias)), dim=1)
        return -h_term -v_term

    def energy(self, v, h):
        v=v.bernoulli()
        h=h.bernoulli()
        return -torch.matmul(v, self.v_bias.t())-torch.matmul(torch.matmul(v, self.Weight.t()),h.t())-torch.matmul(h, self.h_bias.t())
    
    def Energy_GPU2(self, v_list0, h_list0):
        if CUDA:
            n_split=torch.cuda.device_count()
        else:
            n_split=1
        e_list=[]
        m_split=2**6
        for j in range(m_split):
            v_list1=torch.stack(list(v_list0[j*int(len(v_list0)/m_split):(j+1)*int(len(v_list0)/m_split)]))
            h_list1=torch.stack(list(h_list0[j*int(len(h_list0)/m_split):(j+1)*int(len(h_list0)/m_split)]))
            vs=[]
            hs=[]
            for i in range(n_split):
                v_list2=torch.stack(list(v_list1[i*int(len(v_list1)/n_split):(i+1)*int(len(v_list1)/n_split)]))
                h_list2=torch.stack(list(h_list1[i*int(len(h_list1)/n_split):(i+1)*int(len(h_list1)/n_split)]))
                if CUDA:
                    v_list2=v_list2.to(device='cuda:' + str(i)).view(len(v_list2), n_vis)
                    h_list2=h_list2.to(device='cuda:' + str(i)).view(len(h_list2), n_hid)
                else:
                    None
                vs.append(v_list2)
                hs.append(h_list2)
            for i in range(n_split): 
                if CUDA:
                    a=self.v_bias.to(device='cuda:' + str(i)).view(n_vis)
                    b=self.h_bias.to(device='cuda:' + str(i)).view(n_hid)
                    W=self.Weight.to(device='cuda:' + str(i)).view(n_hid, n_vis)
                    e=(-torch.matmul(vs[i].float(), a)-torch.diagonal(torch.matmul(torch.matmul(vs[i].float(), W.t()), hs[i].float().t()))-torch.matmul(hs[i].float(), b)).to('cuda:0')
                    e_list.append(e)
                else:
                    a=self.v_bias.view(n_vis)
                    b=self.h_bias.view(n_hid)
                    W=self.Weight.view(n_hid, n_vis)
                    e=(-torch.matmul(vs[i].float(), a)-torch.diagonal(torch.matmul(torch.matmul(vs[i].float(), W.t()), hs[i].float().t()))-torch.matmul(hs[i].float(), b))
                    e_list.append(e)
        return torch.stack(e_list).view(len(v_list0))
    
    def forward(self, v):
        h = self.v2h(v)
        h = h.bernoulli()
        for _ in range(self.k):
            v_gibbs = self.h2v(h).to(device)
            v_gibbs = v_gibbs.bernoulli()
            h = self.v2h(v_gibbs).to(device)
            h = h.bernoulli()
        return v, v_gibbs
        
from torch.utils.data import Dataset

class CustomDataset(Dataset): 
    def __init__(self, dataset):
        data_x = dataset
        self.x_data = data_x
#         self.y_data = data_y

    # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)
    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
#         y = torch.FloatTensor([self.y_data[idx]])
        return x

def data_to_loader(fullconfigs):
    fulldata=CustomDataset(fullconfigs)
    full_dataset = fulldata
    full_loader = torch.utils.data.DataLoader(full_dataset, batch_size)
    return full_loader

def train_and_get_data(n_hid, model, lr, train_loader):
    # Train and get the new result
    rbm=RBM(n_vis, n_hid, k)
    train_op = optim.SGD(rbm.parameters(), lr, momentum=0.9)
    rbm.train()
    train_loss_list=[]
    val_loss_list=[]
    model_list=[]
    # for epoch in tnrange(n_epochs):
    epoch=0
    for _, (data) in enumerate(train_loader):
        epoch+=1
        data=data.to(device)
        v, v_gibbs = rbm(data.view(-1, n_vis))
        train_loss = torch.mean(rbm.Fv(v)) - torch.mean(rbm.Fv(v_gibbs))
        train_op.zero_grad()
        train_loss.backward()
        train_op.step()
        if epoch in epoch_to_save:
            model_list.append(copy.deepcopy(rbm.cpu().state_dict()))
            rbm.to(device)
            print(epoch)

    return model_list

In [ ]:
# Hyper parameter들을 설정
L=3
n_vis=L*L
k=5
lr=0.1
std=0.5
batch_size=512
epoch_to_save=[2**i for i in range(15)]
n_epochs=epoch_to_save[-1]+1

In [ ]:
lattice1 = - np.ones((L,L))
n0 = 2000
n_delta = 50
n_sample=batch_size*n_epochs+1
mcmax = n_sample+n0


In [ ]:

torch.set_printoptions(precision=10)
for T in [1.9]:
    fullconfigs=Ising_model(lattice1, T, n0, n_delta, n_sample)
    train_loader=data_to_loader(fullconfigs)
    for n_hid in [1,2,3,4,6,8,12]:  
        model0=train_and_get_data(n_hid, 0, lr=lr, train_loader=train_loader)
        with open('{base}/loss_IG/3*3/state_dict/model_n_hid={n_hid}_T={T}_lr={lr}_vol={vol}_std={std}_infinity.pkl'.format(base=base, n_hid=n_hid, T=T, lr=lr, vol=vol, std=std), 'wb') as f:
            pkl.dump(model0, f)


In [ ]:
for vol in [512]:
    for n_hid in [1,2,3,4,6,8,12]:
        v_list_rbm=[]; h_list_rbm=[]
        for s in tqdm(range(2**(n_vis+n_hid))):
            full=decimal_to_binary(s, n_hid+n_vis)[0]
            v=full[:n_vis]; h=full[-n_hid:]
            v_list_rbm.append(v); h_list_rbm.append(h)
        v_list_rbm=torch.stack(v_list_rbm).to(device)
        h_list_rbm=torch.stack(h_list_rbm).to(device)

        v_list_ising=[]
        v_list_ising2=[]
        for s in range(2**n_vis):
            v=decimal_to_binary(s, n_vis)[0]
            v_list_ising.append(np.reshape(v,(3,3))*2-1)
            v_list_ising2.append(v)
        v_list_ising2=torch.stack(v_list_ising2).to(device)      

        for T in [1.9,2.3,5.2]:
            bf_list=np.exp(-Ising_energy(v_list_ising)/T)
            S=entropy(bf_list)
            Pv=torch.tensor(bf_list/sum(bf_list)).to(device)

            dict_GE=[]
            model_dicts=pd.read_pickle('{base}/loss_IG/3*3/state_dict/model_n_hid={n_hid}_T={T}_lr={lr}_vol={vol}_std={std}_infinity.pkl'.format(base=base, L=L, n_hid=n_hid, T=T, lr=lr, vol=vol, std=std))
        
            # for e in range(len(model_dicts['0'])):
            for e in range(len(model_dicts)):

                rbm=RBM(n_vis, n_hid, k)
                rbm.load_state_dict(model_dicts[e])
                Fv=torch.dot(rbm.Fv(v_list_ising2), Pv).detach().cpu().numpy()
                FE=-torch.log(torch.sum(torch.exp(-rbm.Energy_GPU2(v_list_rbm, h_list_rbm)-100))).detach().cpu().numpy()-100
                GE=float((Fv-FE-S))
                dict_GE.append(GE)
                print(T,n_hid,e,GE)
            with open('{base}/loss_IG/3*3/loss/GE_n_hid_{n_hid}_T={T}_lr={lr}_vol={vol}_std={std}_infinity.pkl'.format(base=base, L=L, n_hid=n_hid, T=T, lr=lr, vol=vol, std=std), 'wb') as f:
                pkl.dump(dict_GE, f)